# kohya_ss

## [1] Variables and Functions
**REQUIRED EVERY TIME YOU RUN THIS NOTEBOOK**

In [ ]:
##################################### IMPORT #######################################
import os, sys
import ipywidgets as widgets
import subprocess
from importlib.metadata import version, PackageNotFoundError
from IPython.utils import capture
from IPython.display import clear_output
# import importlib
############################ ENVIRONMENT VARIABLES ################################
os.environ['PIP_ROOT_USER_ACTION'] = 'ignore'
os.environ['PIP_DISABLE_PIP_VERSION_CHECK'] = '1'
os.environ["PYTHONNOUSERSITE"] = "1"
########################### GLOBAL PATHS AND FUNCTION ###########################
root = '/notebooks'
kohya = root + '/kohya_ss'
#################################  FUNCTION ##########################################

#Git Clone
def git_clone(repo_url, repo_name, version=None, recurse_submodules=False, quiet=False, single_branch=False):
    clone_command = ["git", "clone", repo_url, repo_name]
    if recurse_submodules:
        clone_command.append("--recurse-submodules")
    if quiet:
        clone_command.append("--quiet")
    if single_branch:
        clone_command.append("--single-branch")
    try:
        subprocess.run(clone_command, check=True)
        print(f"Successfully cloned {repo_url} into {repo_name}")
    except subprocess.CalledProcessError as e:
        print(f"Error while cloning the repository: {e}")
        return
    if version:
        try:
            os.chdir(repo_name)
            checkout_command = ["git", "checkout", version]
            subprocess.run(checkout_command, check=True)
            print(f"Checked out version: {version}")
        except subprocess.CalledProcessError as e:
            print(f"Error while checking out the version: {e}")
            return
        finally:
            os.chdir("..")

#install package,if package is already exist skip install
def install_package(package_name, version=None):
    """Install the package and display a message only on errors"""
    print(f"Checking {package_name}...")
    result = subprocess.run([sys.executable, "-m", "pip", "show", package_name.split("[")[0]], capture_output=True, text=True)
    
    if package_name.split("[")[0] in result.stdout and (version is None or f"Version: {version}" in result.stdout):
        print(f"{package_name} {version or ''} is already installed. Skipping.")
    else:
        if version:
            print(f"Installing {package_name} {version}...")
            install_command = [sys.executable, "-m", "pip", "install", f"{package_name}=={version}", "--quiet"]
        else:
            print(f"Installing {package_name}...")
            install_command = [sys.executable, "-m", "pip", "install", package_name, "--quiet"]

        try:
            subprocess.run(install_command, check=True, stderr=subprocess.DEVNULL)
            print(f"{package_name} {version or ''} installed successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Error installing {package_name}: {e}")

#unzip
def unziparchive():
    os.chdir(f'{root}/{ziplocation}')
    from zipfile import ZipFile
    zf = ZipFile(f'{zipfilename}.zip', 'r') 
    zf.extractall(f'{root}/{unziplocation}')
    zf.close()

#remove outputs
def deloutput():
    !rm -rf /notebooks/shared-storage/outputs/*

#remove trash files
def delcash():
    !rm -rf ~/.local/share/Trash/*

#remove files
def delfile(iwan2delete):
    if not iwan2delete:
        print("error: 'iwan2delete' is empry, Copy the path of the file or folder and paste it..")
        return  
    # make sure var is not empty
    !rm -rf /notebooks/{iwan2delete}*
    print(f"{iwan2delete} is deleated")

#huggingfacehfilename
def huguploadfile():
    from huggingface_hub import upload_file
    upload_file(path_or_fileobj=f"{root}/{modelpathhug}", path_in_repo=f"{filenamehug}", repo_id=f"{usernamehug}/{reponamehug}", token=f"{tokenhug}")

# Complete message
def completedMessage(): 
    completed = widgets.Button(description='Completed', button_style='success', icon='check')
    print('\n')
    display(completed)

completedMessage()

## [2] Set up kohya_ss and install dependency

### 2.1-Install Kohya_ss
**Only needs to be run once on the first installation**

In [ ]:
print('⏳ downloading...')
os.chdir(root)

repo_url = "https://github.com/bmaltais/kohya_ss.git"
repo_name = "kohya_ss"
version = "v24.1.7"

if not os.path.isdir(repo_name):
    git_clone(repo_url, repo_name, version=version, recurse_submodules=True, quiet=True, single_branch=True)
else:
    print(f"{repo_name} is already exists.")

os.chdir(root)
completedMessage()

### 2.2-Install Dependency
**REQUIRED EVERY TIME YOU RUN THIS NOTEBOOK**

In [ ]:
os.chdir(kohya)
with open('requirements.txt', 'r') as file:
    for line in file:
        if line.strip() and not line.startswith('#'):  # skip comments
            if '==' in line:
                package, version = line.strip().split('==')
                install_package(package, version)
            else:
                install_package(line.strip())

print("All packages processed.")
os.chdir(root)
completedMessage()

### 2.3-Install Tkinter and others
**REQUIRED EVERY TIME YOU RUN THIS NOTEBOOK**

In [ ]:
print('⏳ Installing dependencies...')
with capture.capture_output() as cap:
    !apt -y -q update
    !apt-get -y -q install python3.10-tk
    !pip install -q bitsandbytes==0.43.0
    !pip install onnxruntime-gpu==1.18.0 --extra-index-url https://aiinfra.pkgs.visualstudio.com/PublicPackages/_packaging/onnxruntime-cuda-12/pypi/simple/
    # !pip install -q transformers==4.36.2
completedMessage()

## [3] Run Kohhya_ss

In [ ]:
# If you used stable diffusion webui in this session, re-run 2.2-Install Dependency cell before launch Kohya_ss
os.chdir(kohya)
!python kohya_gui.py --share --headless

## -Other utility for training-

#### Create training folders

In [ ]:
os.chdir(root)
%mkdir -p dataset/{images,logs,outputs,regularization,zip}

### Create Config Toml

In [ ]:
import shutil
import toml
os.chdir(kohya)
shutil.copy("config example.toml", "config.toml")

toml_file_path = 'config.toml'
config = toml.load(toml_file_path)

updates = {
    'model': {'models_dir': '/notebooks/shared-storage/models', 'train_data_dir': '../dataset/images', 'dataset_config': '', 'save_precision': 'fp16'},
    'folders': {'output_dir': '../dataset/outputs', 'reg_data_dir': '', 'logging_dir': ''},
    'basic': {'max_resolution': '1024,1024', 'seed': 0},
    'advanced': {'gradient_checkpointing': 'true', 'state_dir': '', 'vae_dir': ''}
}

for section, values in updates.items():
    if section in config:
        config[section].update(values)
    else:
        config[section] = values

with open(toml_file_path, 'w') as f:
    toml.dump(config, f)
print("Your config.toml created.")
os.chdir(root)
completedMessage()

#### Download file from googledrive

In [ ]:
!pip install gdown -q
os.chdir(f'{root}/dataset/zip')
!gdown "https://drive.google.com/uc?export=download&id={change here}&confirm=t"

#### Unzip archives

In [ ]:
#zip location eg. ziplocation = 'dataset/zip'
ziplocation = 'dataset/zip'
#filename eg. if test.zip, zipfilename = 'test'
zipfilename = 'test'
#unzip location eg. unziplocation = 'dataset/images/test/1_test'
unziplocation = 'dataset/images/test/1_test'

unziparchive()
os.chdir(root)
completedMessage()

### Move Loras

In [ ]:
import shutil
new_path = shutil.move('/notebooks/dataset/outputs/test-000010.safetensors', '/notebooks/shared-storage/lora/sdxl')
print(new_path)

### Delete .ipynb_checkpoints in Training folders

In [ ]:
#Delete ipynb_checkpoints, because they are misrecognized during training.
os.chdir(f'{root}/dataset')
!rm -rf `find -type d -name .ipynb_checkpoints`
os.chdir(root)
completedMessage()

# -Oher function-

### Delete Webui outputs

In [ ]:
deloutput()
os.chdir(root)
completedMessage()

### Delete trash files

In [ ]:
delcash()
os.chdir(root)
completedMessage()

### Delete File or Folder

In [ ]:
# copy and past path
iwan2delete = 'here'
delfile(iwan2delete)
os.chdir(root)
completedMessage()

## Upload merged models etc. to Huggingface

In [ ]:
modelpathhug = 'shared-storage/models/sdxl/test.safetensors'
filenamehug = 'test.safetensors'
usernamehug = 'yourname'
reponamehug = 'yourrepo'
tokenhug = 'hf_yourtoken'
huguploadfile()
completedMessage()